In [10]:
import numpy as np

A = np.array([[2.0, -1.0, 0.0, 0],
              [-1.0, 2.0, -1.0, 0],
              [0.0, -1.0, 2.0, -1],
              [0,0,-1,2]])
b = np.array([1.0, 0, 0,1.0])

**Eliminação de Gauss**

In [11]:
def solve_t(A, b , method = 'backward'):
    n = len(b)
    x = np.zeros(n)
    
    if method == 'forward':
        for i in range(n):
            sum_terms = sum(A[i, j] * x[j] for j in range(i))
            x[i] = (b[i] - sum_terms) / A[i, i]
    
    elif method == 'backward':
        for i in range(n-1, -1, -1):
            sum_terms = sum(A[i, j] * x[j] for j in range(i+1, n))
            x[i] = (b[i] - sum_terms) / A[i, i]
    
    return x

def triangulate(A, b, method = 'upper'):
    n = len(A)

    if method == 'upper':
        for i, j in ((i, j) for i in range(n-1) for j in range(i+1, n)):
            m = A[j, i] / A[i, i]
            A[j] -= m * A[i]
            b[j] -= m * b[i]

    elif method == 'lower':
        for i, j in ((i, j) for i in range(n-1, 0, -1) for j in range(i)):
            m = A[j, i] / A[i, i]
            A[j] -= m * A[i]
            b[j] -= m * b[i]

    return A, b


A1, b1 = triangulate(A.copy(), b.copy(), 'upper')
x1 = solve_t(A1, b1, 'backward')

A2, b2 = triangulate(A.copy(), b.copy(), 'lower')
x2 = solve_t(A2, b2, 'forward')

print(x1)
print(x2)

[1. 1. 1. 1.]
[1. 1. 1. 1.]


**Eliminação de Gauss com pivoteamento parcial**

In [12]:
def partial_pivoting(A, b):
    n = len(A)

    for i in range(n-1):
        max_index = np.argmax(np.abs(A[i:n, i])) + i
        if max_index != i:
            A[[i, max_index]] = A[[max_index, i]]
            b[[i, max_index]] = b[[max_index, i]]

        for j in range(i+1, n):
            m = A[j, i] / A[i, i]
            A[j] -= m * A[i]
            b[j] -= m * b[i]

    return A, b

A1, b1 = partial_pivoting(A.copy(), b.copy())
x = solve_t(A1, b1)

print(x)


[1. 1. 1. 1.]


**Eliminação de Gauss com pivoteamento parcial com escala**

In [13]:
def scaled_partial_pivoting(A, b):
    n = len(A)

    scale = np.max(np.abs(A), axis = 1)

    for i in range(n-1):
        pivot = np.abs(A[i:n, i]) / scale[i:n]
        max_index = np.argmax(pivot) + i
        
        if max_index != i:
            A[[i, max_index]] = A[[max_index, i]]
            b[[i, max_index]] = b[[max_index, i]]
            scale[[i, max_index]] = scale[[max_index, i]]

        for j in range(i+1, n):
            m = A[j, i] / A[i, i]
            A[j] -= m * A[i]
            b[j] -= m * b[i]

    return A, b

A1, b1 = scaled_partial_pivoting(A.copy(), b.copy())
x = solve_t(A1, b1)

print(x)

[1. 1. 1. 1.]


**Determinante de uma Matriz**

In [14]:
def determinant(A):
    n = len(A)
    det = 1.0
    for i in range(n):
        det *= A[i][i]
    return det

print(determinant(A1))

5.0


**Fatoração $LU$**

In [15]:
def LU(A):
    n = len(A)
    U = np.zeros((n, n))
    L = np.identity(n)
    for m in range(n):
        for j in range(m, n):
            U[m, j] = A[m, j] - np.sum(L[m, 0:m] * U[0:m, j])
        for i in range(m + 1, n):
            L[i, m] = (A[i, m] - np.sum(L[i, 0:m] * U[0:m, m])) / U[m, m]
    return L, U

L, U = LU(A.copy())
y = solve_t(L,b, 'forward')
x = solve_t(U, y)

print(x)

[1. 1. 1. 1.]


**Fatoração $LDL^{T}$**

In [16]:
def LDL(A):
    n = len(A)
    L = np.identity(n)
    D = np.zeros(n)

    for j in range(n):
        D[j] = A[j, j] - np.sum(L[j, :j] ** 2 * D[:j])

        for i in range(j + 1, n):
            L[i, j] = (A[i, j] - np.sum(L[i, :j] * L[j, :j] * D[:j])) / D[j]

    return L, D

L, D = LDL(A.copy())

y = solve_t(L, b, 'forward')

x = solve_t(L.T, y/D)

print(x)

[1. 1. 1. 1.]


**Fatoração de Cholesky**

In [17]:
def cholesky(A):
    n = len(A)
    L = np.zeros((n, n))

    for i in range(n):
        L[i, i] = np.sqrt(A[i, i] - np.sum(L[i, :i] ** 2))

        for j in range(i + 1, n):
            L[j, i] = (A[j, i] - np.sum(L[j, :i] * L[i, :i])) / L[i, i]

    return L

L = cholesky(A.copy())

y = solve_t(L, b, 'forward')

x = solve_t(L.T, y)

print(x)


[1. 1. 1. 1.]


**Fatoração de Crout para sistemas tridiagonais**

In [18]:
def crout(A):
    n = len(A)
    L = np.zeros((n, n))
    U = np.identity(n) 

    L[0, 0] = A[0, 0]

    for i in range(1, n):
        U[i-1, i] = A[i-1, i] / L[i-1, i-1] 
        L[i, i-1] = A[i, i-1]
        L[i, i] = A[i, i] - U[i-1, i] * L[i, i-1]

    return L, U

L, U = crout(A.copy())

z = solve_t(L, b, 'forward')
x = solve_t(U, z)

print(x)

[1. 1. 1. 1.]
